In [1]:
import copy
import glob
import pickle
import warnings
from datetime import datetime, timedelta
from itertools import product
import joblib
import os

import cartopy
import cartopy.crs as ccrs
import cartopy.feature
import cartopy.feature as cfeature
import cartopy.feature as cf
import cartopy.io.shapereader as shpreader
import matplotlib as mpl
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import numpy as np
# np.random.seed(42)
import pandas as pd
import shapely.geometry as sgeom
import xarray as xr
from scipy import stats
from scipy.spatial.distance import cdist
from shapely import geometry
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_class_weight

# import cluster_analysis, narm_analysis, som_analysis

In [2]:
# import tensorflow as tf
# tf.test.is_gpu_available(
#     cuda_only=False, min_cuda_compute_capability=None
# )

In [3]:
import keras
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, AveragePooling2D, Dropout, BatchNormalization,SpatialDropout2D
from keras.utils import to_categorical
from keras.layers import LeakyReLU
from keras.layers import ReLU
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

# import visualkeras
# import tensorflow as tf

# ## GLOBAL SEED ##    
# np.random.seed(42)
# tf.random.set_seed(42)

2023-03-16 12:59:19.349676: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:
import optuna
from optuna.samplers import TPESampler
from optuna.integration import TFKerasPruningCallback

optuna.logging.set_verbosity(optuna.logging.WARN)

/glade/work/jhayron/conda-envs/cnn_wr/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load data

## Weather regimes

In [5]:
week1_wr = pd.read_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week1_wr_v3.csv',\
                      index_col = 0, parse_dates = True)
week2_wr = pd.read_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week2_wr_v3.csv',\
                      index_col = 0, parse_dates = True)
week3_wr = pd.read_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week3_wr_v3.csv',\
                      index_col = 0, parse_dates = True)
week4_wr = pd.read_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week4_wr_v3.csv',\
                      index_col = 0, parse_dates = True)
week5_wr = pd.read_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week5_wr_v3.csv',\
                      index_col = 0, parse_dates = True)
week6_wr = pd.read_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week6_wr_v3.csv',\
                      index_col = 0, parse_dates = True)
week7_wr = pd.read_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week7_wr_v3.csv',\
                      index_col = 0, parse_dates = True)
week8_wr = pd.read_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week8_wr_v3.csv',\
                      index_col = 0, parse_dates = True)
week9_wr = pd.read_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week9_wr_v3.csv',\
                      index_col = 0, parse_dates = True)

df_wr = pd.concat([week1_wr,week2_wr,week3_wr,week4_wr,week5_wr,week6_wr,week7_wr,week8_wr,week9_wr],axis=1)
df_wr.columns = ['week1','week2','week3','week4','week5','week6','week7','week8','week9']

df_wr_2 = pd.read_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/weekly_wr_mean_geop_v3.csv',
                     index_col=0,parse_dates=True)

df_wr_2 = df_wr_2.dropna()
df_wr = df_wr.dropna()

In [6]:
distance_wr = pd.read_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/weekly_distance_mean_geop_v3.csv',
                         index_col = 0,parse_dates=True)

In [7]:
df_wr_2[distance_wr>=np.mean(distance_wr.iloc[:,0]) + 1.5 * np.std(distance_wr.iloc[:,0])] = 4

## Input Variables

In [8]:
variables = ['z500']
name_var = ['z500']
units = ['m2/s2']


In [9]:
dic_vars = {}
for var_short, variable,unit in zip(name_var,variables,units):
# for var_short, variable,unit in zip(['sst'],['sst'],['K']):
    path_w_anoms = '/glade/work/jhayron/Weather_Regimes/weekly_anomalies/'
    week1_anoms = xr.open_dataset(f'{path_w_anoms}week1_{variable}_anoms_v3.nc')
    # week1_anoms = week1_anoms.sel(time=df_wr_2.index)
    if variable=='z500':
        week1_anoms = week1_anoms.where(week1_anoms.lat>-30,drop=True)
    # week1_anoms = week1_anoms.sel(time=df_wr.index)
    week1_anoms = week1_anoms.sel(time=df_wr_2.index)
    dic_vars[variable] = week1_anoms
    
    ##########PLOT#####################
#     fig = plt.figure(figsize=(9,7))
#     ax = fig.add_subplot(111,projection=ccrs.PlateCarree(central_longitude=-90+360))
    
#     #     vmax = np.round(np.percentile(abs(week1_anoms[f'{var_short}_anomalies'].values[0]),100),0)
#     #     vmin = -np.round(np.percentile(abs(week1_anoms[f'{var_short}_anomalies'].values[0]),100),0)
    
#     vmax = np.round(np.percentile(abs(week1_anoms[f'{var_short}_anomalies'].values[0]),100),0)
#     vmin = -np.round(np.percentile(abs(week1_anoms[f'{var_short}_anomalies'].values[0]),100),0)
#     # print(vmax,vmin)
#     cs = plt.pcolormesh(week1_anoms.lon,week1_anoms.lat,\
#         week1_anoms[f'{var_short}_anomalies'].values[0],cmap='seismic',
#         transform=ccrs.PlateCarree(),vmin=vmin,vmax=vmax)
#     ax.coastlines(resolution='110m', color='k', linewidth=0.75, zorder=10)
#     ax.margins(x=0, y=0)

#     # ax.set_extent([-179, 179, 10, 90], crs=ccrs.PlateCarree())
#     if variable!='st':
#         plt.title(variable.upper().replace('_','-'))
#     else:
#         plt.title(variable.upper().replace('_','-').replace('ST','TS'))
#     if 'region' in variable:
#         cbar_ax = fig.add_axes([0.25, 0.19, 0.5, 0.0175])
#     else:
#         cbar_ax = fig.add_axes([0.25, 0.3, 0.5, 0.0175])
#     # ticks_1 = [-80, -40, 0, 40, 80]
#     cbar = fig.colorbar(cs, cax=cbar_ax,
#                         orientation='horizontal', extend='both')
#     cbar.ax.tick_params(labelsize=14)
#     cbar.set_label(unit, fontsize=14)
#     plt.savefig(f'/glade/u/home/jhayron/WeatherRegimes/Figures/MapsVariables/{variable}_anomalies_v2.png',bbox_inches='tight')
#     plt.close('all')

# Build models

# Optuna

In [10]:
def create_model(ks,ps,type_pooling,stc,stp,do,bn,md,nfilters,activation):
    num_classes = 5
    
    if activation == 'LeakyReLU':
        activation_conv= LeakyReLU()
    elif activation == 'ReLU':
        activation_conv= ReLU()
        
    padding_type = 'same'
    model = Sequential()
    
    model.add(Conv2D(nfilters, kernel_size=(ks, ks),activation=activation_conv,
        input_shape=X_train.shape[1:],padding=padding_type,strides=stc))
    
    if type_pooling == 'Max':
        model.add(MaxPooling2D((ps, ps),padding=padding_type,strides=stp))
    elif type_pooling == 'Average':
        model.add(AveragePooling2D((ps, ps),padding=padding_type,strides=stp))
        
    model.add(Dropout(do))
    if bn==True:
        model.add(BatchNormalization())
        
    model.add(Conv2D(nfilters*2, (ks, ks), activation=activation_conv,padding=padding_type,strides=stc))
    
    if type_pooling == 'Max':
        model.add(MaxPooling2D((ps, ps),padding=padding_type,strides=stp))
    elif type_pooling == 'Average':
        model.add(AveragePooling2D((ps, ps),padding=padding_type,strides=stp))
        
    model.add(SpatialDropout2D(do))
    if bn==True:
        model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dense(num_classes*md*md, activation=activation_conv))
    model.add(Dense(num_classes*md, activation=activation_conv))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy, 
        optimizer=keras.optimizers.Adam(lr=0.001),metrics=['accuracy'])
    return model

In [11]:
class Objective(object):
    def __init__(self, X_train, y_train, X_val, y_val,
                 path_models, variable, week, d_class_weights):
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.path_models = path_models
        self.variable = variable
        self.week = week
 
    def __call__(self, trial):    
        keras.backend.clear_session()
                
        ks = trial.suggest_categorical('ks',[3,5,7,9,11])
        ps = trial.suggest_categorical('ps',[2,4,6,8])
        type_pooling = trial.suggest_categorical('type_pooling',['Average','Max'])
        stc = trial.suggest_categorical('stc',[1,2,3,4])
        stp = trial.suggest_categorical('stp',[1,2,3,4])
        do = trial.suggest_categorical('do',[0.3,0.4,0.5])
        bn = trial.suggest_categorical('bn',[True,False])
        md = trial.suggest_categorical('md',[2,4,8,16,32])
        nfilters = trial.suggest_categorical('nfilters',[4,8,16,32])
        activation = trial.suggest_categorical('activation',['LeakyReLU','ReLU'])
        bs = trial.suggest_categorical('bs',[16,32,64])
        
        dict_params = {'ks':ks,
                       'ps':ps,
                       'type_pooling':type_pooling,
                       'stc':stc,
                       'stp':stp,
                       'do':do,
                       'bn':bn,
                       'md':md,
                       'nfilters':nfilters,
                       'activation':activation,
                       # 'lr':lr,
                       'bs':bs}
                                              
        # instantiate and compile model
        cnn_model = create_model(dict_params['ks'],
                                 dict_params['ps'],
                                 dict_params['type_pooling'],
                                 dict_params['stc'],
                                 dict_params['stp'],
                                 dict_params['do'],
                                 dict_params['bn'],
                                 dict_params['md'],
                                 dict_params['nfilters'],
                                 dict_params['activation'],
                                 # dict_params['lr'],
                                )
        
        epochs = 50
        earlystop = EarlyStopping(monitor='val_loss', patience=5)
        try:
            os.mkdir(f'{self.path_models}{self.variable}')
        except: pass
        filepath = f'{self.path_models}{self.variable}/model_{self.week}_v7.h5'
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, 
                                     mode='auto',save_weights_only=False)
        h = cnn_model.fit(self.X_train, self.y_train, batch_size=dict_params['bs'],\
            epochs=epochs,verbose=0,validation_data=(self.X_val, self.y_val), \
            callbacks=[checkpoint,earlystop],class_weight = d_class_weights) #TFKerasPruningCallback(trial, "val_loss")

        validation_loss = np.min(h.history['val_loss'])
        return validation_loss

In [12]:
def logging_callback(study, frozen_trial):
    previous_best_value = study.user_attrs.get("previous_best_value", None)
    if previous_best_value != study.best_value:
        study.set_user_attr("previous_best_value", study.best_value)
        print(
            "Trial {} finished with best value: {} and parameters: {}. ".format(
            frozen_trial.number,
            frozen_trial.value,
            frozen_trial.params,
            )
        )

In [13]:
path_models = '/glade/work/jhayron/Weather_Regimes/models/CNN/weights_variables_v8/'

In [ ]:
dic_metrics = {}

for var_short, variable in zip(name_var,variables):
    print('********************************************************************************************')
    print(variable)
    print('********************************************************************************************')
    loss_weeks_model = []
    loss_weeks_persistence = []
    acc_weeks_model = []
    acc_weeks_persistence = []

    for week in ['week1','week2','week3']:
        print(week)
        #### ORGANIZE DATA ####
        week_output_wr = df_wr_2[week].values.astype(int)
        # Make Y categorical
        serie_wr_categorical = to_categorical(week_output_wr,num_classes=5)
        week1_anoms = copy.deepcopy(dic_vars[variable])
        
        # # Scale by min-max
        Min = week1_anoms[f'{var_short}_anomalies'].min(dim='time')
        Max = week1_anoms[f'{var_short}_anomalies'].max(dim='time')
        scaled_x = (week1_anoms[f'{var_short}_anomalies']) / (Max - Min)

        indices = np.arange(len(serie_wr_categorical))
        #Reshape X
        scaled_x = scaled_x.data.reshape(-1, scaled_x.shape[1],scaled_x.shape[2], 1)

        indices_train = np.where(df_wr_2.week2.index.year<=2001)[0]
        indices_val = np.where((df_wr_2.week2.index.year>2001)&(df_wr_2.week2.index.year<=2010))[0]
        indices_test = np.where(df_wr_2.week2.index.year>2010)[0]

        X_test = scaled_x[indices_test]
        y_test = serie_wr_categorical[indices_test]

        X_train = scaled_x[indices_train]
        y_train = serie_wr_categorical[indices_train]

        X_val = scaled_x[indices_val]
        y_val = serie_wr_categorical[indices_val]

        wr_persistence = df_wr_2.week1.values.astype(int)[indices_test]
        serie_wr_persistence_categorical = to_categorical(wr_persistence)
        
        y_train_integers = np.argmax(y_train, axis=1)
        class_weights = compute_class_weight(class_weight='balanced',classes=np.unique(y_train_integers),
                                             y = y_train_integers)
        d_class_weights = dict(enumerate(class_weights))
        
        #### TRAIN ####
        # def print_best_callback(study, trial):
        #     print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

        optimizer_direction = 'minimize'
        number_of_random_points = 25  # random searches to start opt process
        maximum_time = 0.5*60*60  # seconds
        objective = Objective(X_train,y_train,X_val,y_val,path_models,variable,week,d_class_weights)

        # optuna.logging.set_verbosity(optuna.logging.WARNING)
        study = optuna.create_study(direction=optimizer_direction,
                sampler=TPESampler(n_startup_trials=number_of_random_points))

        study.optimize(objective, timeout=maximum_time, gc_after_trial=True,callbacks=[logging_callback])
        
        results_directory = f'/glade/work/jhayron/Weather_Regimes/models/CNN/results_optuna/{week}/'
        # save results
        df_results = study.trials_dataframe()
        df_results.to_pickle(results_directory + 'df_optuna_results_v3.pkl')
        df_results.to_csv(results_directory + 'df_optuna_results_v3.csv')
        #save study
        joblib.dump(study, results_directory + 'optuna_study_v3.pkl')

********************************************************************************************
z500
********************************************************************************************
week1
Trial 0 finished with best value: 0.5757694840431213 and parameters: {'ks': 9, 'ps': 4, 'type_pooling': 'Average', 'stc': 2, 'stp': 1, 'do': 0.5, 'bn': False, 'md': 2, 'nfilters': 4, 'activation': 'LeakyReLU', 'bs': 16}. 
Trial 5 finished with best value: 0.5279024839401245 and parameters: {'ks': 9, 'ps': 6, 'type_pooling': 'Average', 'stc': 1, 'stp': 4, 'do': 0.4, 'bn': True, 'md': 4, 'nfilters': 32, 'activation': 'LeakyReLU', 'bs': 32}. 
Trial 18 finished with best value: 0.5117121338844299 and parameters: {'ks': 3, 'ps': 4, 'type_pooling': 'Max', 'stc': 1, 'stp': 4, 'do': 0.4, 'bn': False, 'md': 16, 'nfilters': 32, 'activation': 'LeakyReLU', 'bs': 16}. 
Trial 28 finished with best value: 0.4995255768299103 and parameters: {'ks': 9, 'ps': 4, 'type_pooling': 'Average', 'stc': 1, 'stp': 4, 'd

# Train and save final models

In [14]:
def create_model(ks,ps,type_pooling,stc,stp,do,bn,md,nfilters,activation):
    num_classes = 5
    
    if activation == 'LeakyReLU':
        activation_conv= LeakyReLU()
    elif activation == 'ReLU':
        activation_conv= ReLU()
        
    padding_type = 'same'
    model = Sequential()
    
    model.add(Conv2D(nfilters, kernel_size=(ks, ks),activation=activation_conv,
        input_shape=X_train.shape[1:],padding=padding_type,strides=stc))
    
    if type_pooling == 'Max':
        model.add(MaxPooling2D((ps, ps),padding=padding_type,strides=stp))
    elif type_pooling == 'Average':
        model.add(AveragePooling2D((ps, ps),padding=padding_type,strides=stp))
        
    model.add(Dropout(do))
    if bn==True:
        model.add(BatchNormalization())
        
    model.add(Conv2D(nfilters*2, (ks, ks), activation=activation_conv,padding=padding_type,strides=stc))
    
    if type_pooling == 'Max':
        model.add(MaxPooling2D((ps, ps),padding=padding_type,strides=stp))
    elif type_pooling == 'Average':
        model.add(AveragePooling2D((ps, ps),padding=padding_type,strides=stp))
        
    model.add(SpatialDropout2D(do))
    if bn==True:
        model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dense(num_classes*md*md, activation=activation_conv))
    model.add(Dense(num_classes*md, activation=activation_conv))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy, 
        optimizer=keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])
    return model

## 1. No weights

In [15]:
path_models = '/glade/work/jhayron/Weather_Regimes/models/CNN/weights_variables_v8/'

In [17]:
dic_metrics = {}

for var_short, variable in zip(name_var,variables):
    print('********************************************************************************************')
    print(variable)
    print('********************************************************************************************')
    loss_weeks_model = []
    loss_weeks_persistence = []
    acc_weeks_model = []
    acc_weeks_persistence = []

    for week in ['week1','week2','week3']:
        print(week)
        #### ORGANIZE DATA ####
        week_output_wr = df_wr_2[week].values.astype(int)
        # Make Y categorical
        serie_wr_categorical = to_categorical(week_output_wr,num_classes=5)
        week1_anoms = copy.deepcopy(dic_vars[variable])
        
        # # Scale by min-max
        Min = week1_anoms[f'{var_short}_anomalies'].min(dim='time')
        Max = week1_anoms[f'{var_short}_anomalies'].max(dim='time')
        scaled_x = (week1_anoms[f'{var_short}_anomalies']) / (Max - Min)

        indices = np.arange(len(serie_wr_categorical))
        #Reshape X
        scaled_x = scaled_x.data.reshape(-1, scaled_x.shape[1],scaled_x.shape[2], 1)

        indices_train = np.where(df_wr_2.week2.index.year<=2001)[0]
        indices_val = np.where((df_wr_2.week2.index.year>2001)&(df_wr_2.week2.index.year<=2010))[0]
        indices_test = np.where(df_wr_2.week2.index.year>2010)[0]

        X_test = scaled_x[indices_test]
        y_test = serie_wr_categorical[indices_test]

        X_train = scaled_x[indices_train]
        y_train = serie_wr_categorical[indices_train]

        X_val = scaled_x[indices_val]
        y_val = serie_wr_categorical[indices_val]

        wr_persistence = df_wr_2.week1.values.astype(int)[indices_test]
        serie_wr_persistence_categorical = to_categorical(wr_persistence)
        
        y_train_integers = np.argmax(y_train, axis=1)
#         class_weights = compute_class_weight(class_weight='balanced',classes=np.unique(y_train_integers),
#                                              y = y_train_integers)
#         d_class_weights = dict(enumerate(class_weights))
        
        ## TRAIN
        results_directory = f'/glade/work/jhayron/Weather_Regimes/models/CNN/results_optuna/{week}/'
        study_optuna = joblib.load(results_directory + 'optuna_study_v3.pkl')
        dict_params = study_optuna.best_params
        print(dict_params)
        keras.backend.clear_session()

        model = create_model(dict_params['ks'],
                             dict_params['ps'],
                             dict_params['type_pooling'],
                             dict_params['stc'],
                             dict_params['stp'],
                             dict_params['do'],
                             dict_params['bn'],
                             dict_params['md'],
                             dict_params['nfilters'],
                             dict_params['activation'],
                            )
        
        batch_size = dict_params['bs']
        epochs = 200

        earlystop = EarlyStopping(monitor='val_loss', patience=5)
        try:
            os.mkdir(f'{path_models}{variable}')
        except: pass
        filepath = f'{path_models}{variable}/model_{week}_v8.h5'
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, 
                                     mode='auto',save_weights_only=False)
        model.fit(X_train, y_train, batch_size=batch_size,\
                  epochs=epochs,verbose=0,validation_data=(X_val, y_val),\
                  callbacks=[checkpoint,earlystop])#,
                  # class_weight = d_class_weights)

        #### EVAL ####

        model.load_weights(filepath)
        model.save(filepath)
        metrics_model = model.evaluate(x=X_test,y=y_test)
        acc_temp = metrics_model[1]
        loss_temp = metrics_model[0]
        acc_persistence = accuracy_score(y_test,serie_wr_persistence_categorical)
        loss_persistence = log_loss(y_test,serie_wr_persistence_categorical)
        print('loss/acc model')
        print(metrics_model)
        print('loss/acc persistence')
        print([loss_persistence,acc_persistence])

        loss_weeks_model.append(loss_temp)
        loss_weeks_persistence.append(loss_persistence)
        acc_weeks_model.append(acc_temp)
        acc_weeks_persistence.append(acc_persistence)

********************************************************************************************
z500
********************************************************************************************
week1
{'activation': 'ReLU', 'bn': True, 'bs': 32, 'do': 0.3, 'ks': 7, 'md': 2, 'nfilters': 4, 'ps': 6, 'stc': 1, 'stp': 2, 'type_pooling': 'Max'}


2023-03-16 13:02:03.020448: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-03-16 13:02:03.021893: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-03-16 13:02:03.060900: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3d:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2023-03-16 13:02:03.060967: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-03-16 13:02:03.266261: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-03-16 13:02:03.266318: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2

18/18 [==============================] - 0s 6ms/step - loss: 0.5397 - accuracy: 0.8180
loss/acc model
[0.5396832823753357, 0.8180180191993713]
loss/acc persistence
[0.0, 1.0]
week2
{'activation': 'LeakyReLU', 'bn': False, 'bs': 32, 'do': 0.4, 'ks': 5, 'md': 4, 'nfilters': 4, 'ps': 4, 'stc': 3, 'stp': 1, 'type_pooling': 'Average'}
18/18 [==============================] - 0s 4ms/step - loss: 1.3646 - accuracy: 0.4396
loss/acc model
[1.3646208047866821, 0.4396396279335022]
loss/acc persistence
[20.661033809077633, 0.4018018018018018]
week3
{'activation': 'ReLU', 'bn': True, 'bs': 32, 'do': 0.3, 'ks': 7, 'md': 8, 'nfilters': 4, 'ps': 4, 'stc': 2, 'stp': 3, 'type_pooling': 'Max'}
18/18 [==============================] - 0s 4ms/step - loss: 1.4992 - accuracy: 0.2919
loss/acc model
[1.4991958141326904, 0.2918919026851654]
loss/acc persistence
[26.5108445863466, 0.23243243243243245]


## 2. Linear weights

In [18]:
path_models = '/glade/work/jhayron/Weather_Regimes/models/CNN/weights_variables_v8_weights_1/'


In [20]:
dic_metrics = {}

for var_short, variable in zip(name_var,variables):
    print('********************************************************************************************')
    print(variable)
    print('********************************************************************************************')
    loss_weeks_model = []
    loss_weeks_persistence = []
    acc_weeks_model = []
    acc_weeks_persistence = []

    for week in ['week1','week2','week3']:
        print(week)
        #### ORGANIZE DATA ####
        week_output_wr = df_wr_2[week].values.astype(int)
        # Make Y categorical
        serie_wr_categorical = to_categorical(week_output_wr,num_classes=5)
        week1_anoms = copy.deepcopy(dic_vars[variable])
        
        # # Scale by min-max
        Min = week1_anoms[f'{var_short}_anomalies'].min(dim='time')
        Max = week1_anoms[f'{var_short}_anomalies'].max(dim='time')
        scaled_x = (week1_anoms[f'{var_short}_anomalies']) / (Max - Min)

        indices = np.arange(len(serie_wr_categorical))
        #Reshape X
        scaled_x = scaled_x.data.reshape(-1, scaled_x.shape[1],scaled_x.shape[2], 1)

        indices_train = np.where(df_wr_2.week2.index.year<=2001)[0]
        indices_val = np.where((df_wr_2.week2.index.year>2001)&(df_wr_2.week2.index.year<=2010))[0]
        indices_test = np.where(df_wr_2.week2.index.year>2010)[0]

        X_test = scaled_x[indices_test]
        y_test = serie_wr_categorical[indices_test]

        X_train = scaled_x[indices_train]
        y_train = serie_wr_categorical[indices_train]

        X_val = scaled_x[indices_val]
        y_val = serie_wr_categorical[indices_val]

        wr_persistence = df_wr_2.week1.values.astype(int)[indices_test]
        serie_wr_persistence_categorical = to_categorical(wr_persistence)
        
        y_train_integers = np.argmax(y_train, axis=1)
        class_weights = compute_class_weight(class_weight='balanced',classes=np.unique(y_train_integers),
                                             y = y_train_integers)
        d_class_weights = dict(enumerate(class_weights))

        ## TRAIN
        results_directory = f'/glade/work/jhayron/Weather_Regimes/models/CNN/results_optuna/{week}/'
        study_optuna = joblib.load(results_directory + 'optuna_study_v3.pkl')
        dict_params = study_optuna.best_params
        print(dict_params)
        keras.backend.clear_session()

        model = create_model(dict_params['ks'],
                             dict_params['ps'],
                             dict_params['type_pooling'],
                             dict_params['stc'],
                             dict_params['stp'],
                             dict_params['do'],
                             dict_params['bn'],
                             dict_params['md'],
                             dict_params['nfilters'],
                             dict_params['activation'],
                            )
        
        batch_size = dict_params['bs']
        epochs = 200

        earlystop = EarlyStopping(monitor='val_loss', patience=5)
        try:
            os.mkdir(f'{path_models}{variable}')
        except: pass
        filepath = f'{path_models}{variable}/model_{week}_v8.h5'
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, 
                                     mode='auto',save_weights_only=False)
        model.fit(X_train, y_train, batch_size=batch_size,\
                  epochs=epochs,verbose=0,validation_data=(X_val, y_val),\
                  callbacks=[checkpoint,earlystop],
                  class_weight = d_class_weights)

        #### EVAL ####

        model.load_weights(filepath)
        model.save(filepath)
        metrics_model = model.evaluate(x=X_test,y=y_test)
        acc_temp = metrics_model[1]
        loss_temp = metrics_model[0]
        acc_persistence = accuracy_score(y_test,serie_wr_persistence_categorical)
        loss_persistence = log_loss(y_test,serie_wr_persistence_categorical)
        print('loss/acc model')
        print(metrics_model)
        print('loss/acc persistence')
        print([loss_persistence,acc_persistence])

        loss_weeks_model.append(loss_temp)
        loss_weeks_persistence.append(loss_persistence)
        acc_weeks_model.append(acc_temp)
        acc_weeks_persistence.append(acc_persistence)

********************************************************************************************
z500
********************************************************************************************
week1
{'activation': 'ReLU', 'bn': True, 'bs': 32, 'do': 0.3, 'ks': 7, 'md': 2, 'nfilters': 4, 'ps': 6, 'stc': 1, 'stp': 2, 'type_pooling': 'Max'}
18/18 [==============================] - 0s 6ms/step - loss: 0.6530 - accuracy: 0.7748
loss/acc model
[0.6529822945594788, 0.7747747898101807]
loss/acc persistence
[0.0, 1.0]
week2
{'activation': 'LeakyReLU', 'bn': False, 'bs': 32, 'do': 0.4, 'ks': 5, 'md': 4, 'nfilters': 4, 'ps': 4, 'stc': 3, 'stp': 1, 'type_pooling': 'Average'}
18/18 [==============================] - 0s 4ms/step - loss: 1.4171 - accuracy: 0.3982
loss/acc model
[1.4170533418655396, 0.3981981873512268]
loss/acc persistence
[20.661033809077633, 0.4018018018018018]
week3
{'activation': 'ReLU', 'bn': True, 'bs': 32, 'do': 0.3, 'ks': 7, 'md': 8, 'nfilters': 4, 'ps': 4, 'stc': 2, 'stp': 3, '